In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import datetime
from scipy import stats

import seaborn as sns
sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'

In [2]:
full_df=pd.read_pickle('../datasets/df_userID.pickle')

In [4]:
full_df[full_df.Race=='20km de Lausanne'].RaceYear.value_counts()

2015    16780
2014    13392
2013    12344
2009    10164
2012    10012
2011     9937
2010     8998
2006     6046
2007     4911
2004     2026
2005     1940
2003     1938
2002      117
2001       94
2000       84
Name: RaceYear, dtype: int64

In [5]:
full_df=full_df[full_df.Weather.notnull()]

In [7]:
full_df[full_df.Race=='20km de Lausanne'].MaxTemp.value_counts()

18.0    33493
19.0    25778
7.0     12344
24.0    10012
Name: MaxTemp, dtype: int64

In [ ]:
full_df.head()

In [ ]:
full_df.Race=full_df.Race.apply(lambda x: 'Lucerne Marathon, Luzern' if x=='SwissCityMarathon - Lucerne, Luzern' else x)

# Categorize weather

In [ ]:
full_df.Weather.value_counts().index

2 -> Sunny
1 -> Cloudy
0 -> bad weather

In [ ]:
dict_favorable_weather={'Partly cloudy':1, 'Clear':2, 'Sunny':2, 'Mist':0, 'Patchy rain possible':1,
       'Cloudy':1, 'Light rain':0, 'Light rain shower':0, 'Moderate rain':0,
       'Overcast':1, 'Fog':0, 'Moderate or heavy rain shower':0,
       'Moderate rain at times':1, 'Light drizzle':0, 'Light snow':0,
       'Patchy light drizzle':0, 'Moderate snow':0, 'Patchy light rain':0,
       'Patchy light rain with thunder':0, 'Freezing fog':0, 'Heavy snow':0,
       'Heavy rain':0, 'Moderate or heavy rain with thunder':0,
       'Moderate or heavy sleet':0, 'Patchy moderate snow':0,
       'Heavy rain at times':0, 'Light sleet':0,
       'Moderate or heavy snow with thunder':0}

In [ ]:
def categorize_weather(x):
    return dict_favorable_weather[x]

In [ ]:
full_df.Weather=full_df.Weather.apply(categorize_weather)

# Races with a lot of years

In [ ]:
distance_to_analyse=21.0975

In [ ]:
good_distances=full_df[(full_df.Distance==distance_to_analyse)]# | (full_df.Distance==21.0975) | (full_df.Distance==42.195)]

In [ ]:
a=good_distances.groupby([good_distances.Race,good_distances.RaceYear]).count().reset_index()

In [ ]:
a=a[['Race','RaceYear','Date']]
a=a.rename(columns={'Date':'Runners'})
a['Years']=1

In [ ]:
race_stats=a.groupby(a.Race).sum()

In [ ]:
race_stats

In [ ]:
race_stats=race_stats[race_stats.Years>5]
race_stats['RunnersPerYear']=race_stats.Runners/race_stats.Years
race_stats=race_stats[race_stats.RunnersPerYear>500]
race_stats

In [ ]:
good_games=race_stats.index
good_games

# Example with a distance

In [ ]:
reduced_df=full_df[full_df.Race.isin(good_games)]

In [ ]:
reduced_df.shape

In [ ]:
reduced_df.RaceYear.value_counts()

In [ ]:
km10=reduced_df[reduced_df.Distance==distance_to_analyse]

In [ ]:
km10.Race.value_counts()

In [ ]:
km10.columns

In [ ]:
km10=km10[['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance',
       'Sex', 'Time', 'Pace',
       'MinTemp', 'MaxTemp', 'Weather', 'RaceID', 'UserID']]

In [ ]:
def add_mean_std_race(x):
    x['MeanRace']=np.mean(x.Time)
    x['StdRace']=np.std(x.Time)/np.sqrt(x.shape[0])
    x['Count']=x.shape[0]
    return x

In [ ]:
def add_mean_multiple_years(x):
    x['MeanMultipleYears']=np.mean(x.Time)
    return x

In [ ]:
km10=km10.groupby(km10.RaceID).apply(add_mean_std_race)
km10=km10.groupby(km10.Race).apply(add_mean_multiple_years)
km10=km10[km10.Sex=='M']

In [ ]:
km10['DiffFromGlobalMean']=km10.MeanRace-km10.MeanMultipleYears
km10.DiffFromGlobalMean=km10.DiffFromGlobalMean.apply(lambda x: x.total_seconds())

In [ ]:
km10['AvgTemp']=(km10.MinTemp+km10.MaxTemp)/2

In [ ]:
km10

In [ ]:
km10.columns

In [ ]:
del final_data

In [ ]:
final_data=km10.groupby(km10.RaceID).first().reset_index()
final_data=final_data[['Race', 'RaceYear','Distance',
       'Weather', 'AvgTemp', 'DiffFromGlobalMean',
       'MeanRace', 'MeanMultipleYears',
        'StdRace', 'Count', 'MinTemp', 'MaxTemp','RaceID' ]]
final_data['CatWeather']=final_data.Weather.apply(lambda x: dict_favorable_weather[x])
final_data

In [ ]:
final_data

In [ ]:
final_data.groupby(final_data.Race).first()

In [ ]:
np.corrcoef(final_data.MinTemp,final_data.DiffFromGlobalMean)

In [ ]:
sns.p

In [ ]:
compact=km10.groupby(km10.Race).first()
compact=compact[['Distance','MeanRace']]
compact

In [ ]:
full_df.Race.value_counts()

In [ ]:
full_df[full_df.Race=='Course de l\'Escalade, Genève'].Distance.value_counts()

In [ ]:
full_df[full_df.notnull()].RaceYear.value_counts()

In [ ]:
full_df[full_df.Race=='20km de Lausanne'].RaceYear.value_counts()